# Day 12

## Part 1

In [1]:
from collections import namedtuple

Point = namedtuple("Point", ("x", "y", "z"))

In [2]:
def cmp(a, b):
    """Return -1, 0 or 1 if a is less, equal or greater than b."""
    return (a > b) - (a < b)

In [3]:
cmp(2, 3)

-1

In [4]:
cmp(3, 3)

0

In [5]:
cmp(4, 3)

1

In [6]:
def compute_gravity(a, b):
    """Compute the gravity vector applied to a by b."""
    return Point(
        cmp(b.x, a.x),
        cmp(b.y, a.y),
        cmp(b.z, a.z),
    )

In [7]:
compute_gravity(
    Point(0, 0, 0),
    Point(1, 1, 1),
)

Point(x=1, y=1, z=1)

In [8]:
compute_gravity(
    Point(1, 2, 3),
    Point(1, -1, 0),
)

Point(x=0, y=-1, z=-1)

In [9]:
def sum_point(a, b):
    """Summ two points."""
    return Point(
        a.x + b.x,
        a.y + b.y,
        a.z + b.z,
    )

In [10]:
sum_point(
    Point(1, 2, 3),
    Point(-1, 4, 0),
)

Point(x=0, y=6, z=3)

In [11]:
def compute_velocity(point, velocity, map_):
    """Return the new velocity for point, applying gravity for all points in map_"""
    for other in map_:
        if other == point:
            continue

        gravity = compute_gravity(point, other)
        velocity = sum_point(velocity, gravity)
    
    return velocity

In [12]:
map_ = [
    Point(-1, 0, 2),
    Point(2, -10, -7),
    Point(4, -8, 8),
    Point(3, 5, -1),
]

compute_velocity(
    map_[0],
    Point(0, 0, 0),
    map_,
)  # should return 3, -1, -1

Point(x=3, y=-1, z=-1)

In [13]:
map_ = [
    Point(2, -1, 1),
    Point(3, -7, -4),
    Point(1, -7, 5),
    Point(2, 2, 0),
]

compute_velocity(
    map_[0],
    Point(3, -1, -1),
    map_,
)  # should return 3, -2, -2

Point(x=3, y=-2, z=-2)

In [49]:
def compute_new_step(map_, velocities):
    """Compute the next positions."""
    result_map = []
    result_velocities = []
    for point, velocity in zip(map_, velocities):
        new_velocity = compute_velocity(point, velocity, map_)
        new_point = sum_point(point, new_velocity)

        result_map.append(new_point)
        result_velocities.append(new_velocity)
        
    return result_map, result_velocities

In [58]:
map_ = [
    Point(-1, 0, 2),
    Point(2, -10, -7),
    Point(4, -8, 8),
    Point(3, 5, -1),
]
compute_new_step(
    map_,
    [Point(0, 0, 0)] * 4,
)

([Point(x=2, y=-1, z=1),
  Point(x=3, y=-7, z=-4),
  Point(x=1, y=-7, z=5),
  Point(x=2, y=2, z=0)],
 [Point(x=3, y=-1, z=-1),
  Point(x=1, y=3, z=3),
  Point(x=-3, y=1, z=-3),
  Point(x=-1, y=-3, z=1)])

In [16]:
def compute_energy(point):
    return abs(point.x) + abs(point.y) + abs(point.z)

In [17]:
def compute_total_energy(map_, velocities):
    return sum(
        compute_energy(p) * compute_energy(v) for p, v in zip(map_, velocities)
    )

In [18]:
map_ = [
    Point(-1, 0, 2),
    Point(2, -10, -7),
    Point(4, -8, 8),
    Point(3, 5, -1),
]
velocities = [Point(0, 0, 0)] * 4

for i in range(10):
    map_, velocities = compute_new_step(map_, velocities)

map_, velocities

([Point(x=2, y=1, z=-3),
  Point(x=1, y=-8, z=0),
  Point(x=3, y=-6, z=1),
  Point(x=2, y=0, z=4)],
 [Point(x=-3, y=-2, z=1),
  Point(x=-1, y=1, z=3),
  Point(x=3, y=2, z=-3),
  Point(x=1, y=-1, z=-1)])

In [19]:
compute_total_energy(map_, velocities)  # should return 179

179

Actual data:

In [57]:
%%time
map_ = [
    Point(3, -6, 6),
    Point(10, 7, -9),
    Point(-3, -7, 9),
    Point(-8, 0, 4),
]
velocities = [Point(0, 0, 0)] * 4

for i in range(1000):
    map_, velocities = compute_new_step(map_, velocities)
    
compute_total_energy(map_, velocities)

CPU times: user 39.8 ms, sys: 0 ns, total: 39.8 ms
Wall time: 41.6 ms


6849

## Part 2

We want to known when an already seen state will append.
But first we need to better performance.

In [118]:
from collections import defaultdict
from itertools import combinations
import time

def compute_new_step_p2(map_, velocities):
    """Compute the next positions."""
    result_map = []
    result_velocities = []
    gravities = defaultdict(lambda: [])

    for a, b in combinations(map_, 2):
        g_ab = compute_gravity(a, b)
        g_ba = Point(- g_ab.x, - g_ab.y, - g_ab.z)
        
        gravities[a].append(g_ab)
        gravities[b].append(g_ba)
    
    for p, v in zip(map_, velocities):
        nv = Point(
            v.x + sum(g.x for g in gravities[p]),
            v.y + sum(g.y for g in gravities[p]),
            v.z + sum(g.z for g in gravities[p]),
        )
        result_map.append(
            Point(
                p.x + nv.x,
                p.y + nv.y,
                p.z + nv.z
            )
        )
        result_velocities.append(nv)
        
    return result_map, result_velocities

In [121]:
%%timeit

map_ = [
    Point(3, -6, 6),
    Point(10, 7, -9),
    Point(-3, -7, 9),
    Point(-8, 0, 4),
]
velocities = [Point(0, 0, 0)] * 4

for i in range(1000):
    map_, velocities = compute_new_step_p2(map_, velocities)
    
compute_total_energy(map_, velocities)  # should still be 6849

29.6 ms ± 331 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


The perf is exactly the same :(

In [71]:
map_ = [
    Point(-8, -10, 0),
    Point(5, 5, 10),
    Point(2, -7, 3),
    Point(9, -8, -3),
]
velocities = [Point(0, 0, 0)] * 4
for i in range(100):
    map_, velocities = compute_new_step_p2(map_, velocities)
    
compute_total_energy(map_, velocities)  # should be 1940

1940

In [86]:
def find_repetition(map_):
    map_ = tuple(map_)
    velocities = tuple([Point(0, 0, 0)] * len(map_))
    
    states = set([(map_, velocities)])
    
    i = 0
    while True:
        i += 1
        map_, velocities = compute_new_step_p2(map_, velocities)
        map_ = tuple(map_)
        velocities = tuple(velocities)

        if (map_, velocities) in states:
            break
        else:
            maps.add(map_)
            maps_velocities.add(velocities)

    return i, map_, velocities

In [87]:
%%time

map_ = [
    Point(-1, 0, 2),
    Point(2, -10, -7),
    Point(4, -8, 8),
    Point(3, 5, -1),
]
i, map_, velocities = find_repetition(map_)
i

CPU times: user 116 ms, sys: 3 µs, total: 116 ms
Wall time: 115 ms


2772

In [88]:
%%time

map_ = [
    Point(-8, -10, 0),
    Point(5, 5, 10),
    Point(2, -7, 3),
    Point(9, -8, -3),
]
i, map_, velocities = find_repetition(map_)
i

KeyboardInterrupt: 

In [73]:
map_ = (
    Point(3, -6, 6),
    Point(10, 7, -9),
    Point(-3, -7, 9),
    Point(-8, 0, 4),
)
velocities = (
    Point(0, 0, 0),
    Point(0, 0, 0),
    Point(0, 0, 0),
    Point(0, 0, 0),
)

maps = set([map_])   
maps_velocities = set([velocities])

i = 0
while True:
    i += 1
    map_, velocities = compute_new_step(map_, velocities)
    map_ = tuple(map_)
    velocities = tuple(velocities)
    if map_ in maps and velocities in maps_velocities:
        break
    else:
        maps.add(map_)
        maps_velocities.add(velocities)

i

KeyboardInterrupt: 